## Evaluating the predictive accuracy of physics emulators without hyperparameter fine tuning

In [ ]:
from tqdm.notebook import tqdm

import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from duqling_interface import DuqlingInterface

from plot_performance import heatmap

In [ ]:
from sklearn.cross_decomposition      import PLSRegression
from sklearn.linear_model             import LassoLars, ElasticNet
from sklearn.ensemble                 import ExtraTreesRegressor, GradientBoostingRegressor
from sklearn.svm                      import SVR
from sklearn.gaussian_process         import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, ConstantKernel, WhiteKernel
from sklearn.pipeline                 import Pipeline
from sklearn.preprocessing            import StandardScaler

models = [
    PLSRegression(n_components=1),
    LassoLars(random_state=42),
    ElasticNet(max_iter=10_000, random_state=42),
    ExtraTreesRegressor(random_state=42),
    GradientBoostingRegressor(random_state=42),
    SVR(kernel='rbf'),
    Pipeline([
        ('scale', StandardScaler()),
        ('gpr',   GaussianProcessRegressor(
            kernel = ConstantKernel(1.0, (1e-2, 1e2))
                    * Matern()
                    + WhiteKernel(1e-3, (1e-6, 1e1)),
            n_restarts_optimizer = 3,
            random_state = 42,
            normalize_y = True
        ))
    ])
]

In [ ]:
duq = DuqlingInterface()

univariate_funcs = duq.list_functions(response_type='uni').fname

As Kelin pointed out, latin hypercube sampling will produce distinct distributions of data based on the random seed.

In [ ]:
duqling_funcs = duq.list_functions(response_type='uni').fname
model_names = [model.__class__.__name__ for model in models]

df_mse = pd.DataFrame(columns=duqling_funcs, index=model_names)

In [ ]:
for fname in tqdm(duqling_funcs):
    if fname == 'cube3_rotate':
        continue
    X_trn, y_trn = duq.generate_data(fname, 1000, seed=41)
    X_tst, y_tst = duq.generate_data(fname, 1000, seed=42)
    for model in models:
        model.fit(X_trn, y_trn)
        y_pred = model.predict(X_tst)
        df_mse.at[model.__class__.__name__, fname] = mean_squared_error(y_pred, y_tst)

In [ ]:
import xarray as xr
from pathlib import Path
import pickle

metrics     = ["test_rmse", "sigma", "r_val"]
model_names = ['pls', 'lassolars', 'elasticnet', 'xt', 'gbr', 'svr', 'gpr']

arr = np.full((len(metrics), len(model_names), len(univariate_funcs)), np.nan)

for j, model in enumerate(model_names):
    for k, func in enumerate(univariate_funcs):
        pkl = Path('models', model, func, 'cv_no_fold_results.pkl')
        if not pkl.exists(): # can remove when cube3_rotate gets fixed
            continue
        with pkl.open("rb") as fh:
            data = pickle.load(fh)
        for i, m in enumerate(metrics):
            arr[i, j, k] = data[m]

summary = xr.DataArray(
    arr,
    coords={"metric": metrics, "model": model_names, "function": univariate_funcs},
    dims=["metric", "model", "function"]
)

def metric_df(metric: str) -> pd.DataFrame:
    """Return a model and function DataFrame for a single metric."""
    return summary.sel(metric=metric).to_pandas()

df_std  = metric_df('sigma')
df_rmse_tune = metric_df('test_rmse')

cols_to_drop = [
    # 'circuit', 'cantilever_S', 'banana', 'cube3_rotate', 'steel_column',
    'const_fn', 'const_fn3', 'const_fn15', 'cube3_rotate'
]

In [ ]:
df_rmse = df_mse.map(np.sqrt)

In [ ]:
df_rmse[['dms_radial', 'ripples', 'Gfunction6', 'Gfunction12', 'Gfunction18', 'onehundred']]

In [ ]:
fig1 = heatmap(df_rmse.drop(cols_to_drop, axis=1), "Test RMSE")
fig2 = heatmap((df_rmse/df_std).drop(cols_to_drop, axis=1), "Test RMSE / \u03C3")
fig3 = heatmap((df_rmse/df_std)[(df_rmse/df_std).drop(cols_to_drop, axis=1)>1].drop(cols_to_drop, axis=1), "(Test RMSE / \u03C3) > 1")

df_filtered = (df_rmse/df_std)[(df_rmse/df_std).drop(cols_to_drop, axis=1)>1].drop(cols_to_drop, axis=1)
drop_cols = df_filtered.columns[df_filtered.isnull().all()]
fig4 = heatmap(df_filtered.drop(drop_cols, axis=1), "(Test RMSE / \u03C3) > 1")

In [ ]:
hh = df_rmse[['dms_radial', 'ripples', 'Gfunction6', 'Gfunction12', 'Gfunction18', 'onehundred']].copy()
hh.index = ['pls', 'lassolars', 'elasticnet', 'xt', 'gbr', 'svr', 'gpr']

In [ ]:
heatmap(hh, "Test RMSE")

In [ ]:
df_rmse_tune.index = ['pls', 'lassolars', 'elasticnet', 'xt', 'gbr', 'svr', 'gpr']
df_rmse.index = ['pls', 'lassolars', 'elasticnet', 'xt', 'gbr', 'svr', 'gpr']

In [ ]:
diff_df = df_rmse_tune - df_rmse
diff_df_filtered = diff_df[diff_df>0]
outlier_cols = diff_df.columns[diff_df_filtered.isnull().all()]
heatmap(diff_df_filtered.drop(outlier_cols, axis=1), 'Failed CV Tuning')
# heatmap(diff_df, 'RMSE Impact From Tuning')

In [ ]:
df_rmse_tune['Gfunction18']

In [ ]:
df_rmse['Gfunction18']